In [1]:
import cadquery as cq
from jupyter_cadquery.cadquery import show, Assembly, Part, Faces, Edges
from jupyter_cadquery import set_defaults, set_sidecar, get_defaults

from jupyter_cadquery.mate_assembly import Mate, MAssembly, MAssemblyRenderer, jc_show

set_defaults(axes=True, axes0=True, edge_accuracy=0.01, timeit=False)
set_sidecar("CadQuery")

Overwriting auto display for cadquery Workplane and Shape


# Hexapod 
## 1 Components

In [2]:
thickness = 2
height = 40
width = 65
length = 100
diam = 4
tol = 0.05

In [3]:
x1, x2 = 0.63, 0.87
base_holes = {
    "right_back": (-x1*length, -x1*width), "right_middle": (0, -x2*width), "right_front": ( x1*length, -x1*width),
    "left_back":  (-x1*length,  x1*width), "left_middle":  (0,  x2*width), "left_front":  ( x1*length,  x1*width),
}
stand_dist = {"front_stand": 0.75 * length, "back_stand": -0.8 * length}

base = (cq.Workplane()
    .ellipse(length, width).pushPoints(list(base_holes.values())).circle(diam / 2 + tol)
    .extrude(thickness)
)

stand_cutout = cq.Workplane().box(thickness + 2 * tol, width / 2 + 2 * tol, thickness * 2)
front_cutout = cq.Workplane().box(length / 3, 2 * (width + tol), 2 * thickness)

base = (base
    .cut(front_cutout.translate((length, 0, 0)))
    .cut(stand_cutout.translate((stand_dist["back_stand"], 0, thickness)))
    .cut(stand_cutout.translate((stand_dist["front_stand"], 0, thickness)))
    .faces(">X").edges("not |Y").fillet(width / 5)
)

In [4]:
stand = cq.Workplane().box(height, width / 2 + 10, thickness)
inset = cq.Workplane().box(thickness, width / 2, thickness)
backing = cq.Workplane("ZX").polyline([(10,0), (0,0), (0, 10)]).close().extrude(thickness)

stand = (stand
    .union(inset.translate(( (height + thickness) / 2, 0, 0)))
    .union(inset.translate((-(height + thickness) / 2, 0, 0)))
    .union(backing.translate((-height / 2, 0, thickness / 2)))
    .union(backing.rotate((0, 0, 0), (0, 1, 0), -90).translate((height / 2, 0, thickness / 2)))
)

In [5]:
l1, l2 = 50, 80
pts = [( 0,  0), ( 0, height/2), (l1, height/2 - 5), (l2, 0)]
upper_leg_hole = (l2 - 10, 0)

upper_leg = (cq.Workplane()
    .polyline(pts).mirrorX()
    .pushPoints([upper_leg_hole]).circle(diam/2 + tol).extrude(thickness)
    .edges("|Z and (not <X)").fillet(4)
)

axle = (cq.Workplane("XZ", origin=(0, height/2 + thickness + tol, thickness/2))
    .circle(diam/2).extrude(2 * (height/2 + thickness + tol))
)

upper_leg = upper_leg.union(axle)

In [6]:
w, l1, l2 = 15, 20, 120
pts = [( 0,  0), ( l1, w), (l2, 0)]
lower_leg_hole = (l1 - 10, 0)

lower_leg = (cq.Workplane()
    .polyline(pts).mirrorX()
    .pushPoints([lower_leg_hole]).circle(diam/2 + tol)
    .extrude(thickness)
    .edges("|Z").fillet(5)
)

In [7]:
c = show(base, 
    stand.translate((0,100,0)), 
    upper_leg.translate((-100,-100,0)), 
    lower_leg.translate((0,-100,0)),
)

Done, using side car 'CadQuery'


## 2 Assembly Definition

In [8]:
leg_names = ("left_back", "left_middle", "left_front", "right_back", "right_middle", "right_front")
stand_names = ("front_stand", "back_stand")

def create_hexapod():
    # Some little shortcuts
    L = lambda *args: cq.Location(cq.Vector(*args))
    C = lambda *args: cq.Color(*args)

    # Leg assembly

    leg = MAssembly(upper_leg, name="upper", color=C("orange"))
    leg.add(        lower_leg, name="lower", color=C("orange"), loc=L(80,0,0))

    # Hexapod assembly

    hexapod = MAssembly(base,  name="bottom",      color=C("gray"))
    hexapod.add(        base,  name="top",         color=C(0.9, 0.9, 0.9), loc=L(-2*length,   0, 0))
    hexapod.add(        stand, name="front_stand", color=C(0.5, 0.8, 0.9), loc=L(0,        -100, 0))
    hexapod.add(        stand, name="back_stand",  color=C(0.5, 0.8, 0.9), loc=L(0,         100, 0))

    for i, name  in enumerate(leg_names):
        hexapod.add(leg, name=name, loc=L(100, -50*(i-2.5), 0))

    return hexapod

## 3.1 Mate Definitions 

In [9]:
hexapod = create_hexapod()

# another little shortcut
M = lambda *args: Mate(hexapod.find(*args))

leg_angles = {"right_back":   105, "right_middle":  90, "right_front":   75,
              "left_back":   -105, "left_middle":  -90, "left_front":   -75}

hexapod.mate("bottom_mate", "bottom", M("bottom", ("faces", ">Z")), is_origin=True)
hexapod.mate("top_mate",    "top",    M("top",    ("faces", "<Z")).xr(180).zt(-(height + 2 * tol)), is_origin=True)

for name in stand_names:
    hexapod.mate(f"bottom_{name}", "bottom", M("bottom", ("faces", "<Z"), ("wires", (stand_dist[name], 0))).xr(180).zr(180 if "front" in name else 0))
    hexapod.mate(f"{name}_lower",  name,     M(name,     ("faces", "<X")).zr(90), is_origin=True)

for name, pnt in base_holes.items():
    hexapod.mate(f"bottom_{name}", "bottom", M("bottom", ("faces", "<Z"), ("wires", pnt)).xr(180).zr(leg_angles[name]))

for name in leg_names:
    lf, uf = (">Z", "<Z") if "left" in name else ("<Z", ">Z")
    hexapod.mate(f"{name}_lower_hinge",  f"{name}>lower", M(f"{name}>lower", ("faces", lf), ("wires", lower_leg_hole)), is_origin=True)
    hexapod.mate(f"{name}_upper_hinge",  name,            M(name,            ("faces", uf), ("wires", upper_leg_hole)).zr(-75))
    hexapod.mate(f"{name}_bottom_hinge", name,            M(name,            ("faces", "<Y")), is_origin=True)

jc_show(hexapod, mates=True)

/opt/anaconda/envs/cq-ocp/lib/python3.7/site-packages/cadquery/utils.py:21: FutureWarning: Default walue of centerOption will change in the next relase to ProjectedOrigin
  FutureWarning,


Done, using side car 'CadQuery'


In [10]:
hexapod.relocate()
jc_show(hexapod, mates=True)

Done, using side car 'CadQuery'


## 3.2 Assembly

In [11]:
for leg in leg_names:
    hexapod.assemble(f"{leg}_bottom_hinge", f"bottom_{leg}")
    hexapod.assemble(f"{leg}_lower_hinge", f"{leg}_upper_hinge")

for stand_name in stand_names:
    hexapod.assemble(f"{stand_name}_lower", f"bottom_{stand_name}")

hexapod.assemble("top_mate", "bottom_mate")

d = jc_show(hexapod, mates=True)

Done, using side car 'CadQuery'



# = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = 

In [12]:
r = MAssemblyRenderer(hexapod)
view = r.view()
view

Renderer(camera=CombinedCamera(far=3787.706659761525, height=600.0, mode='orthographic', orthoFar=3787.7066597…

In [15]:
import math
import time
g = view.scene.children[10]
try:
    for i in range(2400):
        g.children[4].children[1].rotateZ(math.pi/24)#
        g.children[4].rotateZ(math.pi/12)
        time.sleep(0.05)
except:
    pass